In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

from src.dataset import get_datasets
from src.feature import create_features, split
# Adjust the import below to match your project structure.
from src.benchmark import extend_by_predictions_and_samples

# Prepare the data for checking candidate features.
data = create_features(get_datasets()[0])
data.index = pd.to_datetime(data.index)

best_features = [
    "Pumped storage generation",
    "Solar",
    "Wind offshore",
    "temperature_2m",
    "wind_speed_100m",
    "hour",
    "dayofweek",
    "dayofyear",
    "ma_1_days",
    "ma_1_days_pumped_storage_generation",
    "ma_2_days",
    "ma_2_days_pumped_storage_generation",
    "ma_3_days",
    "ma_4_days_pumped_storage_generation",
    "ma_5_days_pumped_storage_generation",
    "ma_6_days",
    "ma_7_days",
    "ma_8_days_pumped_storage_generation",
    "ma_9_days_pumped_storage_generation",
    "ma_10_days",
    "ma_11_days",
    "ma_12_days_pumped_storage_generation",
    "ma_13_days",
    "ma_14_days_pumped_storage_generation",
]

# Define the candidate features.
candidate_features = [
    'Hydro',
    'Pumped storage generation', 'Solar',
    'Wind offshore', 'Wind onshore', 'temperature_2m', 'precipitation',
    'wind_speed_100m', 'direct_radiation', 'hour', 'dayofweek', 'dayofyear'
]

for i in range(1, 15):
    candidate_features.append(f'ma_{i}_days')
    candidate_features.append(f'ma_{i}_days_pumped_storage_generation')
def objective(trial):
    # --------------------------------------------------
    # 1. Feature Selection
    # --------------------------------------------------
    # Let Optuna decide whether to include each candidate feature.
    # selected_features = []
    # for feature in candidate_features:
    #     use_feature = trial.suggest_categorical(feature, [True, False])
    #     if use_feature and feature in data.columns:
    #         selected_features.append(feature)
    # # If no feature was selected, fall back to using all candidate features.
    # if len(selected_features) == 0:
    #     selected_features = candidate_features

    # --------------------------------------------------
    # 2. Hyperparameter Search Space for XGBoost
    # --------------------------------------------------
    # Sample early_stopping_rounds separately so it can be passed to fit.
    params = {
        'base_score': trial.suggest_float('base_score', 0.0, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 100),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 250)
    }

    # --------------------------------------------------
    # 3. Rolling Forecast Loop with Early Stopping
    # --------------------------------------------------
    merged_df, _ = get_datasets()
    # The split function is assumed to return (train, eval, test, benchmark, …)
    train_df, eval_df, test_df, benchmark_df, _, _, _ = split(merged_df)

    # Combine available training data.
    training_set = pd.concat([train_df, eval_df, test_df])
    # Benchmarking set for rolling forecast.
    benchmarking_set = benchmark_df

    # Ensure the datasets are sorted by their datetime index.
    training_set = training_set.sort_index()
    benchmarking_set = benchmarking_set.sort_index()

    WINDOW_SIZE = 24  # Forecast window size (24 hours)
    rmse_scores = []

    # Loop over forecast windows (starting at index 24 to ensure an initial training period).
    for window_start in range(24, len(benchmarking_set) - WINDOW_SIZE, WINDOW_SIZE):
        # The actual target values for the current forecast window.
        y_actual = benchmarking_set.iloc[window_start: window_start + WINDOW_SIZE]["Price"]

        # Extend the dataset with training data and all benchmark data up to the current window.
        dataset_extended = pd.concat([training_set, benchmarking_set.iloc[:window_start]])
        next_timestamp = dataset_extended.index[-1] + pd.DateOffset(hours=1)
        if next_timestamp != y_actual.index[0]:
            print(f"\nSkipping prediction for {next_timestamp} due to missing entries.\n"
                  "--------------------------------------------------------------")
            continue

        # Optionally extend the dataset with predictions/samples.
        dataset_extended_ps = extend_by_predictions_and_samples(
            dataset_extended, dataset_extended.index[-1]
        )

        # Create features on the extended dataset.
        dataset_extended_features = create_features(dataset_extended_ps)
        print(y_actual.index, "\n##############################################")
        print(dataset_extended_features.index)

        # Check that the forecast window rows are present.
        if not set(y_actual.index).issubset(dataset_extended_features.index):
            print("Forecast window indices are missing in features, skipping.")
            continue

        selected_features = best_features # uncomment for feature selection

        # Split the extended dataset into training and forecast portions.
        # Training data: all data except the last WINDOW_SIZE hours.
        train_data = dataset_extended_features.iloc[:-WINDOW_SIZE]
        # Create an evaluation set (the latter 20% of the training data) for early stopping.
        split_idx = int(0.8 * len(train_data))
        X_train = train_data.iloc[:split_idx][selected_features]
        y_train = train_data.iloc[:split_idx]["Price"]
        X_eval = train_data.iloc[split_idx:][selected_features]
        y_eval = train_data.iloc[split_idx:]["Price"]

        # The prediction set is the forecast window.
        X_predict = dataset_extended_features.reindex(y_actual.index)[selected_features]
        if X_predict.isnull().any().any():
            print("Missing features in forecast window, skipping.")
            continue

        model = xgb.XGBRegressor(
            **params,
            objective='reg:squarederror',
            eval_metric='rmse',
            random_state=42,
            n_jobs=2
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_eval, y_eval)],
            verbose=False
        )

        preds = model.predict(X_predict)
        rmse = mean_squared_error(y_actual, preds)  ** 0.5
        rmse_scores.append(rmse)

    if len(rmse_scores) == 0:
        return float('inf')
    avg_rmse = np.mean(rmse_scores)
    return avg_rmse

# =============================================================================
# Run the Optuna Study
# =============================================================================
n_trials = 100
pbar = tqdm(total=n_trials, desc="Optuna Trials")

def progress_callback(study, trial):
    pbar.update(1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials, n_jobs=2, callbacks=[progress_callback])

print("Best trial:")
best_trial = study.best_trial
print(f"  Average RMSE: {best_trial.value}")
print("  Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")


Optuna Trials:   0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-02-10 18:02:54,324] A new study created in memory with name: no-name-3dc96b82-a3f6-48da-bfa9-cfac22eed736


DatetimeIndex(['2025-01-07 00:00:00', '2025-01-07 01:00:00',
               '2025-01-07 02:00:00', '2025-01-07 03:00:00',
               '2025-01-07 04:00:00', '2025-01-07 05:00:00',
               '2025-01-07 06:00:00', '2025-01-07 07:00:00',
               '2025-01-07 08:00:00', '2025-01-07 09:00:00',
               '2025-01-07 10:00:00', '2025-01-07 11:00:00',
               '2025-01-07 12:00:00', '2025-01-07 13:00:00',
               '2025-01-07 14:00:00', '2025-01-07 15:00:00',
               '2025-01-07 16:00:00', '2025-01-07 17:00:00',
               '2025-01-07 18:00:00', '2025-01-07 19:00:00',
               '2025-01-07 20:00:00', '2025-01-07 21:00:00',
               '2025-01-07 22:00:00', '2025-01-07 23:00:00'],
              dtype='datetime64[ns]', freq=None) 
##############################################
DatetimeIndex(['2018-10-14 23:00:00', '2018-10-15 00:00:00',
               '2018-10-15 01:00:00', '2018-10-15 02:00:00',
               '2018-10-15 03:00:00', '2018-10-1

[I 2025-02-10 18:03:22,279] Trial 0 finished with value: inf and parameters: {'base_score': 0.4023037496547617, 'learning_rate': 0.05185699894775817, 'max_depth': 37, 'n_estimators': 430, 'gamma': 4.999567530723389, 'min_child_weight': 2, 'subsample': 0.6106009796830527, 'colsample_bylevel': 0.7498447346062492, 'max_delta_step': 9, 'early_stopping_rounds': 154}. Best is trial 0 with value: inf.


DatetimeIndex(['2025-02-05 00:00:00', '2025-02-05 01:00:00',
               '2025-02-05 02:00:00', '2025-02-05 03:00:00',
               '2025-02-05 04:00:00', '2025-02-05 05:00:00',
               '2025-02-05 06:00:00', '2025-02-05 07:00:00',
               '2025-02-05 08:00:00', '2025-02-05 09:00:00',
               '2025-02-05 10:00:00', '2025-02-05 11:00:00',
               '2025-02-05 12:00:00', '2025-02-05 13:00:00',
               '2025-02-05 14:00:00', '2025-02-05 15:00:00',
               '2025-02-05 16:00:00', '2025-02-05 17:00:00',
               '2025-02-05 18:00:00', '2025-02-05 19:00:00',
               '2025-02-05 20:00:00', '2025-02-05 21:00:00',
               '2025-02-05 22:00:00', '2025-02-05 23:00:00'],
              dtype='datetime64[ns]', freq=None) 
##############################################
DatetimeIndex(['2018-10-14 23:00:00', '2018-10-15 00:00:00',
               '2018-10-15 01:00:00', '2018-10-15 02:00:00',
               '2018-10-15 03:00:00', '2018-10-1

[W 2025-02-10 18:03:34,810] Trial 1 failed with parameters: {'base_score': 0.8690630983701515, 'learning_rate': 0.03509740310006119, 'max_depth': 73, 'n_estimators': 300, 'gamma': 3.4317980118367766, 'min_child_weight': 1, 'subsample': 0.6173326801267456, 'colsample_bylevel': 0.8278007323873928, 'max_delta_step': 2, 'early_stopping_rounds': 130} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/boh/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/zy/2cj896n535n1dkk6yxct9vhw0000gn/T/ipykernel_5614/3048886667.py", line 124, in objective
    dataset_extended_features = create_features(dataset_extended_ps)
  File "/Users/boh/code/LSDI-Project-/hand_in/src/feature.py", line 174, in create_features
    df.dropna(inplace=True)
  File "/Users/boh/Library/Python/3.9/lib/python/site-packages/pandas/core/frame.py", line 6433, in dropna
    resu

KeyboardInterrupt: 

In [1]:
import optuna
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from prophet import Prophet
import pandas as pd
import numpy as np

from src.dataset import get_datasets
from src.feature import create_features, split
from src.benchmark import extend_by_predictions_and_samples

# =============================================================================
# Prepare the Data and Candidate Features
# =============================================================================

# Use an initial dataset to check available features.2
initial_dataset = get_datasets()[0]
data = create_features(initial_dataset)
data.index = pd.to_datetime(data.index)

# Define candidate features.
candidate_features = [
    'Hydro',
    'Pumped storage generation',
    'Solar',
    'Wind offshore',
    'Wind onshore',
    'temperature_2m',
    'precipitation',
    'wind_speed_100m',
    'direct_radiation',
    'hour',
    'dayofweek',
    'dayofyear'
]

# Append moving-average features for various windows.
for i in range(1, 15):
    candidate_features.append(f'ma_{i}_days')
    candidate_features.append(f'ma_{i}_days_pumped_storage_generation')

# Remove duplicates (if any) while preserving order.
candidate_features = list(dict.fromkeys(candidate_features))


# =============================================================================
# Precompute the Train/Test and Forecast Windows Outside the Objective
# =============================================================================

# Get and split the data only once.
merged_df, _ = get_datasets()
train_df, eval_df, test_df, benchmark_df, _, _, _ = split(merged_df)

# Combine available training data and ensure sorting.
training_set = pd.concat([train_df, eval_df, test_df]).sort_index()
benchmarking_set = benchmark_df.sort_index()

# Define the forecast window size.
WINDOW_SIZE = 24  # (24 hours)

# Precompute the forecast windows.
# For each forecast window (rolling every 24 hours), we build the extended dataset,
# run the (IO heavy) extend_by_predictions_and_samples and create_features functions,
# and store the results for use in every trial.
precomputed_forecast_windows = []
for window_start in range(WINDOW_SIZE, len(benchmarking_set) - WINDOW_SIZE, WINDOW_SIZE):
    # The actual target values for the current forecast window.
    y_actual = benchmarking_set.iloc[window_start: window_start + WINDOW_SIZE]["Price"]

    # Create the dataset up to the current forecast window.
    dataset_extended = pd.concat([training_set, benchmarking_set.iloc[:window_start]])
    next_timestamp = dataset_extended.index[-1] + pd.DateOffset(hours=1)
    if next_timestamp != y_actual.index[0]:
        print(f"Skipping window at {window_start} due to missing entries (expected {next_timestamp} but got {y_actual.index[0]})")
        continue

    # Extend the dataset with predictions/samples.
    dataset_extended_ps = extend_by_predictions_and_samples(dataset_extended, dataset_extended.index[-1])

    # Create features on the extended dataset.
    dataset_extended_features = create_features(dataset_extended_ps)

    # Check if all forecast window indices are present.
    if not set(y_actual.index).issubset(dataset_extended_features.index):
        print(f"Forecast window indices are missing for window starting at {window_start}, skipping.")
        continue

    precomputed_forecast_windows.append({
        "window_start": window_start,
        "y_actual": y_actual,
        "features": dataset_extended_features
    })

print(f"Precomputed {len(precomputed_forecast_windows)} forecast windows out of {((len(benchmarking_set)-WINDOW_SIZE)//WINDOW_SIZE)} possible windows.")


# =============================================================================
# Objective Function for Optuna
# =============================================================================
def objective(trial):
    # -------------------------------
    # 1. Trial-specific Feature Selection
    # -------------------------------
    selected_features = []
    for feature in candidate_features:
        use_feature = trial.suggest_categorical(feature, [True, False])
        if use_feature and feature in data.columns:
            selected_features.append(feature)
    # If no feature was selected, fall back to using all candidate features.
    if len(selected_features) == 0:
        selected_features = candidate_features
    # Remove duplicates (if any)
    selected_features = list(dict.fromkeys(selected_features))

    rmse_scores = []

    # -------------------------------
    # 2. Rolling Forecast Loop (using precomputed forecast windows)
    # -------------------------------
    for window in precomputed_forecast_windows:
        y_actual = window["y_actual"]
        features_df = window["features"]

        # Split the features: use all but the last WINDOW_SIZE rows for training.
        X_train_df = features_df.iloc[:-WINDOW_SIZE]
        try:
            # Select only the trial-selected features for forecasting.
            X_predict_df = features_df.loc[y_actual.index, selected_features]
        except KeyError as e:
            print("Error selecting forecast features:", e)
            continue

        # Skip this window if any forecast feature is missing.
        if X_predict_df.isnull().any().any():
            print("Missing features in forecast window, skipping.")
            continue

        # Prepare the training dataframe in Prophet's expected format.
        X_train_prophet = (
            X_train_df[["Price"] + selected_features]
            .rename(columns={"Price": "y"})
            .reset_index()
            .rename(columns={"index": "ds"})
            .dropna()
        )
        prophet_X_predict = X_predict_df.reset_index().rename(columns={"index": "ds"})

        # Initialize and fit the Prophet model.
        model = Prophet()
        for reg in selected_features:
            model.add_regressor(reg)
        model.fit(X_train_prophet)

        # Generate the forecast.
        forecast = model.predict(prophet_X_predict)
        prophet_forecast = forecast["yhat"]

        # Compute the RMSE for the forecast window.
        rmse = mean_squared_error(y_actual, prophet_forecast) ** 0.5
        rmse_scores.append(rmse)

    if len(rmse_scores) == 0:
        return float('inf')
    avg_rmse = np.mean(rmse_scores)
    return avg_rmse


# =============================================================================
# Run the Optuna Study
# =============================================================================

n_trials = 1
pbar = tqdm(total=n_trials, desc="Optuna Trials")

def progress_callback(study, trial):
    pbar.update(1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials, n_jobs=-1, callbacks=[progress_callback])
pbar.close()

print("Best trial:")
best_trial = study.best_trial
print(f"  Average RMSE: {best_trial.value}")
print("  Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")


Importing plotly failed. Interactive plots will not work.


Precomputed 30 forecast windows out of 30 possible windows.


Optuna Trials:   0%|          | 0/1 [00:00<?, ?it/s]

[I 2025-02-09 23:28:57,247] A new study created in memory with name: no-name-ceb71363-8a34-450a-863f-ecbd050f0c7b
23:29:00 - cmdstanpy - INFO - Chain [1] start processing
23:29:56 - cmdstanpy - INFO - Chain [1] done processing
23:29:59 - cmdstanpy - INFO - Chain [1] start processing
23:30:45 - cmdstanpy - INFO - Chain [1] done processing
23:30:49 - cmdstanpy - INFO - Chain [1] start processing
23:31:35 - cmdstanpy - INFO - Chain [1] done processing
23:31:39 - cmdstanpy - INFO - Chain [1] start processing
23:32:34 - cmdstanpy - INFO - Chain [1] done processing
23:32:38 - cmdstanpy - INFO - Chain [1] start processing
23:33:32 - cmdstanpy - INFO - Chain [1] done processing
23:33:36 - cmdstanpy - INFO - Chain [1] start processing
23:34:36 - cmdstanpy - INFO - Chain [1] done processing
23:34:40 - cmdstanpy - INFO - Chain [1] start processing
23:35:30 - cmdstanpy - INFO - Chain [1] done processing
23:35:34 - cmdstanpy - INFO - Chain [1] start processing
23:36:26 - cmdstanpy - INFO - Chain [1

Best trial:
  Average RMSE: 85.20296017046316
  Best hyperparameters:
    Hydro: True
    Pumped storage generation: False
    Solar: True
    Wind offshore: True
    Wind onshore: False
    temperature_2m: False
    precipitation: True
    wind_speed_100m: False
    direct_radiation: True
    hour: False
    dayofweek: True
    dayofyear: True
    ma_1_days: False
    ma_1_days_pumped_storage_generation: True
    ma_2_days: False
    ma_2_days_pumped_storage_generation: False
    ma_3_days: False
    ma_3_days_pumped_storage_generation: True
    ma_4_days: True
    ma_4_days_pumped_storage_generation: True
    ma_5_days: True
    ma_5_days_pumped_storage_generation: True
    ma_6_days: False
    ma_6_days_pumped_storage_generation: False
    ma_7_days: False
    ma_7_days_pumped_storage_generation: True
    ma_8_days: True
    ma_8_days_pumped_storage_generation: True
    ma_9_days: True
    ma_9_days_pumped_storage_generation: True
    ma_10_days: False
    ma_10_days_pumped_storage_

In [ ]:
import numpy as np
import pandas as pd
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

from src.dataset import get_datasets
from src.feature import create_features, split
from src.benchmark import extend_by_predictions_and_samples

# ------------------------------------------------------------------------------
# Prepare the data just once for checking which features exist in columns
# ------------------------------------------------------------------------------
data = create_features(get_datasets()[0])
data.index = pd.to_datetime(data.index)

# ------------------------------------------------------------------------------
# We specify the set of candidate features from which Optuna will pick
# ------------------------------------------------------------------------------
candidate_features = [
    'Hydro',
    'Pumped storage generation', 'Solar',
    'Wind offshore', 'Wind onshore', 'temperature_2m', 'precipitation',
    'wind_speed_100m', 'direct_radiation', 'hour', 'dayofweek', 'dayofyear'
]
for i in range(1, 15):
    candidate_features.append(f'ma_{i}_days')
    candidate_features.append(f'ma_{i}_days_pumped_storage_generation')


def objective(trial):
    # --------------------------------------------------------------------------
    # 1. Feature Selection via Optuna
    #    Each candidate feature is either included (True) or excluded (False).
    # --------------------------------------------------------------------------
    selected_features = []
    for feature in candidate_features:
        use_feature = trial.suggest_categorical(feature, [True, False])
        if use_feature and feature in data.columns:
            selected_features.append(feature)

    # If no feature was selected, return a large number so that Optuna discards it.
    if len(selected_features) == 0:
        return float('inf')

    # --------------------------------------------------------------------------
    # 2. Prepare the rolling forecast data
    #
    #    Using split() from your code:
    #    Assuming it returns: (train_df, eval_df, test_df, benchmark_df, …)
    #    We combine train_df + test_df (skip eval_df), then do the rolling forecast 
    #    on the benchmark_df.
    # --------------------------------------------------------------------------
    merged_df, _ = get_datasets()
    train_df, eval_df, test_df, benchmark_df, _, _, _ = split(merged_df)

    # Combine all training data (excluding eval_df).
    training_set = pd.concat([train_df, test_df])
    benchmarking_set = benchmark_df

    # Make sure data is sorted by datetime index
    training_set = training_set.sort_index()
    benchmarking_set = benchmarking_set.sort_index()

    WINDOW_SIZE = 24  # The size of each forecast window, in hours
    rmse_scores = []

    # --------------------------------------------------------------------------
    # 3. Rolling Forecast Loop (24-hour steps)
    # --------------------------------------------------------------------------
    # Start from index = 24 so we have some initial data for training.
    for window_start in range(24, len(benchmarking_set) - WINDOW_SIZE, WINDOW_SIZE):
        y_actual = benchmarking_set.iloc[window_start : window_start + WINDOW_SIZE]["Price"]

        # Extend the dataset with all training data + partial benchmark data 
        dataset_extended = pd.concat([training_set, benchmarking_set.iloc[:window_start]])

        # Make sure the forecast start matches the next timestamp
        next_timestamp = dataset_extended.index[-1] + pd.DateOffset(hours=1)
        if next_timestamp != y_actual.index[0]:
            print(
                f"\nSkipping prediction for {next_timestamp} due to missing entries.\n"
                "--------------------------------------------------------------"
            )
            continue

        # Optionally extend by predictions/samples if your pipeline requires it
        dataset_extended_ps = extend_by_predictions_and_samples(
            dataset_extended, dataset_extended.index[-1]
        )

        # Create features on the extended dataset
        dataset_extended_features = create_features(dataset_extended_ps)

        # Check that the forecast window rows exist in our feature set
        if not set(y_actual.index).issubset(dataset_extended_features.index):
            print("Forecast window indices are missing in features, skipping.")
            continue

        # ----------------------------------------------------------------------
        # Split the extended dataset into training and forecast sets
        # ----------------------------------------------------------------------
        # Here we train on everything except the last WINDOW_SIZE samples
        train_data = dataset_extended_features.iloc[:-WINDOW_SIZE]
        X_train = train_data[selected_features]
        y_train = train_data["Price"]

        # The 24-hour forecast window
        X_predict = dataset_extended_features.reindex(y_actual.index)[selected_features]

        # If any feature is missing data for the forecast, skip
        if X_predict.isnull().any().any():
            print("Missing features in forecast window, skipping.")
            continue

        # ----------------------------------------------------------------------
        # 4. Train a simple Linear Regression on the entire train_data
        # ----------------------------------------------------------------------
        model = LinearRegression()
        model.fit(X_train, y_train)

        # ----------------------------------------------------------------------
        # 5. Make predictions for the next 24 hours and compute RMSE
        # ----------------------------------------------------------------------
        preds = model.predict(X_predict)
        rmse = mean_squared_error(y_actual, preds, squared=False)
        rmse_scores.append(rmse)

    # If no forecasts were made, return a large error
    if len(rmse_scores) == 0:
        return float('inf')
    
    # Return the mean RMSE across all forecast windows
    return np.mean(rmse_scores)

# ------------------------------------------------------------------------------
# Run the Optuna Study to find the best subset of features
# ------------------------------------------------------------------------------
n_trials = 300
pbar = tqdm(total=n_trials, desc="Optuna Trials")

def progress_callback(study, trial):
    pbar.update(1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials, n_jobs=1, callbacks=[progress_callback])

print("Best trial:")
best_trial = study.best_trial
print(f"  Average RMSE: {best_trial.value}")
print("  Best feature subset:")
for key, value in best_trial.params.items():
    if value:  # Only list the features that are True
        print(f"    {key}")
